In [41]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyreadr
from sklearn.preprocessing import scale
from scipy.stats import t

from numpy.linalg import slogdet
from scipy.special import digamma, polygamma, gammaln

In [66]:
rw_price = pd.read_csv("rw_price.csv")
rw_returns = pd.read_csv("rw_returns.csv")
log_rw_returns = pd.read_csv("log_rw_returns.csv")

In [67]:
winLen = 200
Nday = len(log_rw_returns)
print(Nday)

S_cov = np.corrcoef(scale(log_rw_returns).T)
S_cov

999


array([[1.        , 0.34982739, 0.32699292, 0.27549521, 0.34886486],
       [0.34982739, 1.        , 0.37785361, 0.33174046, 0.32484628],
       [0.32699292, 0.37785361, 1.        , 0.33200448, 0.34886946],
       [0.27549521, 0.33174046, 0.33200448, 1.        , 0.32947489],
       [0.34886486, 0.32484628, 0.34886946, 0.32947489, 1.        ]])

### Checking Graph operators from 
A Unified Framework for Structured Graph Learning via Spectral Constraints

https://jmlr.org/papers/volume21/19-276/19-276.pdf

In [42]:
def w_from_L(M: np.ndarray) -> np.ndarray:
    M = np.asarray(M)
    if M.ndim != 2 or M.shape[0] != M.shape[1]:
        raise ValueError("M must be a square matrix")

    n = M.shape[0]
    w = np.empty(n * (n - 1) // 2, dtype=M.dtype)
    k = 0
    for i in range(n - 1):            # i = 0 .. n-2
        for j in range(i + 1, n):     # j = i+1 .. n-1 (strict upper)
            w[k] = -M[i, j]
            k += 1
    return w

def L_from_w(w: np.ndarray) -> np.ndarray:
    w = np.asarray(w).ravel()
    k = w.size
    n = int((1 + np.sqrt(1 + 8 * k)) / 2)
    if n * (n - 1) // 2 != k:
        raise ValueError("Invalid length of w: must be n(n-1)/2 for some integer n.")

    Lw = np.zeros((n, n), dtype=float)

    # Fill the strict upper triangle row by row, backwards like in C++
    for i in range(n - 2, -1, -1):
        j = n - i - 1
        # Equivalent of Eigen: Lw.row(i).tail(j) = -w.head(k).tail(j)
        Lw[i, -j:] = -w[:k][-j:]
        k -= j

    # Make symmetric: copy upper triangle into lower
    Lw = Lw + Lw.T

    # Adjust diagonal: subtract column sums
    colsum = Lw.sum(axis=0)
    np.fill_diagonal(Lw, Lw.diagonal() - colsum)

    return Lw

def W_init(M):
    W0 = w_from_L(M)
    W0[W0 < 0] = 0
    return W0

def L_star(Y):

    Y = np.asarray(Y)
    
    if Y.ndim != 2 or Y.shape[0] != Y.shape[1]:
        raise ValueError("Y must be a square matrix.")
    p = Y.shape[0]
    Lstar = np.zeros(int(p*(p-1)/2))

    for i in range(1,p+1):

        for j in range(1,p+1):
           
            if (i > j):
                k = int( (i - j) + ((j - 1)/2)*(2*p - j) ) 
                Lstar[k-1] = Y[i-1,i-1] - Y[i-1, j-1] - Y[ j-1, i-1] + Y[ j-1, j-1]


    return Lstar

def A_from_w(w):

    w = np.asarray(w, dtype=float).ravel()
    k = w.size

    # Infer n from k = n*(n-1)/2
    n_float = (1 + np.sqrt(1 + 8*k)) / 2
    n = int(n_float)
    if n*(n-1)//2 != k:
        raise ValueError(f"len(w)={k} is not a triangular number; expected k=n*(n-1)/2.")

    A = np.zeros((n, n), dtype=float)

    # Indices of strict upper triangle in row-major order
    iu = np.triu_indices(n, k=1)
    A[iu] = w            # fill upper triangle
    A[(iu[1], iu[0])] = w  # mirror to lower triangle
    return A

def w_from_A(M):
    
    N = M.shape[1]
    k = N * (N - 1) // 2
    w = np.zeros(k)
    l = 0

    for i in range(N - 1):
        for j in range(i + 1, N):
            w[l] = M[i, j]
            l += 1
    return w


def A_star(Y):

    Y = np.asarray(Y)
    
    if Y.ndim != 2 or Y.shape[0] != Y.shape[1]:
        raise ValueError("Y must be a square matrix.")
    p = Y.shape[0]
    Astar = np.zeros(int(p*(p-1)/2))

    for i in range(1,p+1):

        for j in range(1,p+1):
           
            if (i > j):
                k = int( (i - j) + ((j - 1)/2)*(2*p - j) ) 
                Astar[k-1] = Y[i-1, j-1] + Y[ j-1, i-1]


    return Astar

def D_star(w: np.ndarray) -> np.ndarray:
    w = np.asarray(w).ravel()
    dStar = L_star(np.diag(w))
    return  dStar

def D_from_w(w: np.ndarray) -> np.ndarray:
    """
    Return the column-wise sums of A(w).
    Equivalent to Eigen: A(w).colwise().sum()
    """
    M = A_from_w(w)               # shape (m, p)
    return M.sum(axis=0)   # shape (p,)

In [43]:
log_rw_returns = pd.read_csv("log_rw_returns.csv")

S_cov = np.corrcoef(scale(log_rw_returns).T)
print("S_cov")
print(S_cov)

S_prec = np.linalg.pinv(S_cov)
print("S_prec")
print(S_prec)

S_cov
[[1.         0.34982739 0.32699292 0.27549521 0.34886486]
 [0.34982739 1.         0.37785361 0.33174046 0.32484628]
 [0.32699292 0.37785361 1.         0.33200448 0.34886946]
 [0.27549521 0.33174046 0.33200448 1.         0.32947489]
 [0.34886486 0.32484628 0.34886946 0.32947489 1.        ]]
S_prec
[[ 1.27263721 -0.24947558 -0.19281752 -0.11937063 -0.25633942]
 [-0.24947558  1.32359962 -0.28620275 -0.21902718 -0.17092178]
 [-0.19281752 -0.28620275  1.32304704 -0.21528053 -0.23040203]
 [-0.11937063 -0.21902718 -0.21528053  1.25090233 -0.22424173]
 [-0.25633942 -0.17092178 -0.23040203 -0.22424173  1.29921337]]


In [44]:
wL = w_from_L(S_prec)
print("w from L")
print(wL)

w = W_init(S_prec)
print("w")
print(w)

Lw = L_from_w(w)
print("Lw from w")
print(Lw) 

Lstar = L_star(Lw)
print("Lstar from Lw")
print(Lstar) 

Aw = A_from_w(w)
print('Aw from w')
print(Aw)

wA = w_from_A(Aw)
print('w from A')
print(wA)

Astar = A_star(Aw)
print('Astar from Aw')
print(Astar)

Dstar = D_star(w)
print('Dstar from w')
print(Dstar)

Dw = D_from_w(w)
print('Dw from w')
print(Dw)

w from L
[0.24947558 0.19281752 0.11937063 0.25633942 0.28620275 0.21902718
 0.17092178 0.21528053 0.23040203 0.22424173]
w
[0.24947558 0.19281752 0.11937063 0.25633942 0.28620275 0.21902718
 0.17092178 0.21528053 0.23040203 0.22424173]
Lw from w
[[ 0.81800315 -0.24947558 -0.19281752 -0.11937063 -0.25633942]
 [-0.24947558  0.9256273  -0.28620275 -0.21902718 -0.17092178]
 [-0.19281752 -0.28620275  0.92470283 -0.21528053 -0.23040203]
 [-0.11937063 -0.21902718 -0.21528053  0.77792007 -0.22424173]
 [-0.25633942 -0.17092178 -0.23040203 -0.22424173  0.88190495]]
Lstar from Lw
[2.24258162 2.12834103 1.83466448 2.21258694 2.42273564 2.14160172
 2.14937582 2.13318397 2.26741184 2.10830848]
Aw from w
[[0.         0.24947558 0.19281752 0.11937063 0.25633942]
 [0.24947558 0.         0.28620275 0.21902718 0.17092178]
 [0.19281752 0.28620275 0.         0.21528053 0.23040203]
 [0.11937063 0.21902718 0.21528053 0.         0.22424173]
 [0.25633942 0.17092178 0.23040203 0.22424173 0.        ]]
w from A


## Code From Paper

In [88]:
log_rw_returns = pd.read_csv("log_rw_returns.csv")

winLen = 200

X = scale(log_rw_returns[0:winLen].to_numpy())

# number of nodes
p = X.shape[1] # Predictors
k = 1 # Clusters

# number of observations
T_n = X.shape[0]
sigma_e = np.exp(0.1) # Student T Distribution Standard Dev. 
alpha = 2/(T_n*sigma_e)
beta = 2*np.log(sigma_e)/T_n


S_cov = np.corrcoef(X.T)
S_prec = np.linalg.pinv(S_cov)

w = W_init(S_prec)
A0 = A_from_w(w)
A0 = A0/np.sum(A0,axis=1,keepdims=True)
w = w_from_A(A0)

Lw = L_from_w(w)
Aw = A_from_w(w)

eigVals,eigVecs = np.linalg.eigh(Lw)
U = eigVecs[:,p-k:]

a0 = 1
a = np.repeat(a0, p*(p-1)/2) 
w_lagged = np.repeat(0, p*(p-1)/2)

Theta = Lw.copy()
Phi_n = np.zeros((p,p))

eta = 1e-8

u_n = w - a*w_lagged
mu_vec = np.repeat(0,p*(p-1)/2) # u_n ADMM dual

d = 1 # degree Constraint
z_n = np.repeat(0,p) # Degree Dual 

V = np.zeros((p,k))# p x k  {p = predictors, k = clusters}

rho = 1                 # ADMM hyperparameter. 
gamma = 10
mu = 2
tau = 2

# residual vectors
primal_lap_residual = np.array([], dtype=float)
primal_deg_residual = np.array([], dtype=float)
dual_residual       = np.array([], dtype=float)

lagrangian = np.array([], dtype=float)
eta_seq = np.array([], dtype=float)

elapsed_time = np.array([], dtype=float)

In [89]:
x = X[0:winLen,:]

In [ ]:
def L_update( wn, Phi_n, rho, p, k):

    ''' 
    L_n: Laplace Matrix (Ln = Diag(Wn1) - Wn = Lwn) Laplacian operator “A Unified Framework for Structured Graph Learning via Spectral Constraints”
    Phi_n: ADMM Dual Variable 
    rho: ADMM Hyperparameter
    p: Number of time series
    k: Clusters
    '''
    # Reference: Equation 16
    
    Y = L_from_w(wn) + Phi_n/rho

    eigVals,eigVecs = np.linalg.eigh(Y)
    eigenVecs = eigVecs[:, ::-1]       # take first (p-k) eigenvectors
    eigenVals = eigVals[::-1]  
    U = eigenVecs[:, :p-k]         # take first (p-k) eigenvectors
    R = np.diag(eigenVals[:p-k]  )
    I = np.eye(R.shape[0]) 
    LnUpdate = 0.5 * U @ ( R + np.sqrt(  R**2 + 4/rho * I  ) ) @ U.T



    return LnUpdate


LnUpdate = L_update( w, Phi_n, rho, p, k)
print(LnUpdate)

[[ 1.44585728 -0.37923887 -0.44282582 -0.26005171 -0.36374089]
 [-0.37923887  1.48845796 -0.39252868 -0.37573339 -0.34095702]
 [-0.44282582 -0.39252868  1.47060587 -0.31423994 -0.32101143]
 [-0.26005171 -0.37573339 -0.31423994  1.25513167 -0.30510663]
 [-0.36374089 -0.34095702 -0.32101143 -0.30510663  1.33081597]]


In [91]:

# Reference
# https://zouyuxin.github.io/Note/EMtDistribution.pdf
# https://shoichimidorikawa.github.io/Lec/ProbDistr/t-e.pdf
def fit_multivariate_t(
    X,
    nu_init=10.0,
    max_iter=200,
    tol=1e-6,
    fix_nu=None,
    jitter=1e-6,
    verbose=False,
):
    """
    Estimate (mu, Sigma, nu) for a multivariate Student's t via EM.

    Parameters
    ----------
    X : array, shape (n_samples, n_features)
        Data.
    nu_init : float
        Initial degrees of freedom (ignored if fix_nu is not None).
    max_iter : int
        Maximum EM iterations.
    tol : float
        Relative tolerance on log-likelihood for convergence.
    fix_nu : float or None
        If set, keep nu fixed at this value.
    jitter : float
        Diagonal jitter multiplier for numerical stability.
    verbose : bool
        Print progress if True.

    Returns
    -------
    mu : array, shape (p,)
    Sigma : array, shape (p, p)
    nu : float
    history : dict with 'loglik'
    """
    X = np.asarray(X)
    n, p = X.shape

    # Initialize
    mu = X.mean(axis=0)
    # sample covariance (unbiased=False), add tiny jitter
    centered0 = X - mu
    Sigma = centered0.T @ centered0 / n
    Sigma += np.eye(p) * (jitter * np.trace(Sigma) / p + 1e-12)
    nu = float(nu_init if fix_nu is None else fix_nu)

    def mahal_sq(Sigma, X, mu):
        # δ_i = (x_i - mu)^T Sigma^{-1} (x_i - mu)
        L = np.linalg.cholesky(Sigma)
        Y = np.linalg.solve(L, (X - mu).T)   # shape (p, n)
        return np.sum(Y * Y, axis=0)         # shape (n,)

    def loglik(mu, Sigma, nu):
        # log-likelihood of multivariate t
        sgn, logdet = slogdet(Sigma)
        if sgn <= 0:
            return -np.inf
        delta = mahal_sq(Sigma, X, mu)
        term1 = gammaln((nu + p) / 2) - gammaln(nu / 2)
        term2 = - (p / 2) * np.log(nu * np.pi) - 0.5 * logdet
        term3 = - ((nu + p) / 2) * np.log1p(delta / nu)
        return np.sum(term1 + term2 + term3)

    ll_old = -np.inf
    history = {'loglik': []}

    for it in range(1, max_iter + 1):
        # E-step
        delta = mahal_sq(Sigma, X, mu)
        w = (nu + p) / (nu + delta)  # E[lambda_i | x_i]
        eloglam = digamma((nu + p) / 2) - np.log((nu + delta) / 2)  # E[log lambda_i | x_i]

        # M-step: mu (weighted mean)
        sumw = w.sum()
        mu = (w[:, None] * X).sum(axis=0) / sumw

        # M-step: Sigma
        Xc = X - mu
        Sigma = (Xc.T * w) @ Xc / n
        # stabilize
        Sigma += np.eye(p) * (jitter * np.trace(Sigma) / p + 1e-12)

        # M-step: nu (solve for root of dQ/dnu = 0) unless fixed
        if fix_nu is None:
            # f(nu) = log(nu/2) - psi(nu/2) + 1 + (1/n) * sum(E[log lambda_i] - E[lambda_i]) = 0
            c = (eloglam.mean() - w.mean())

            def f(nu_):
                return np.log(nu_ / 2.0) - digamma(nu_ / 2.0) + 1.0 + c

            def fprime(nu_):
                return 1.0 / nu_ - 0.5 * polygamma(1, nu_ / 2.0)

            # Newton update with simple guarding
            nu_new = max(nu, 2.01)  # keep > 2 so covariance exists
            for _ in range(30):
                val = f(nu_new)
                der = fprime(nu_new)
                step = val / der
                nu_new = nu_new - step
                if nu_new < 2.01:
                    nu_new = 2.01
                if abs(step) / nu_new < 1e-6:
                    break
            nu = float(nu_new)
        else:
            nu = float(fix_nu)

        # Evaluate log-likelihood and check convergence
        ll = loglik(mu, Sigma, nu)
        history['loglik'].append(ll)
        if verbose:
            print(f"iter {it:3d}: ll={ll:.6f}, nu={nu:.4f}")

        if np.isfinite(ll_old):
            if abs(ll - ll_old) / (abs(ll_old) + 1e-12) < tol:
                break
        ll_old = ll

    return mu, Sigma, nu, history







# X: (n_samples, n_features) data

mu, Sigma, nu, history = fit_multivariate_t( x, nu_init=30.0, max_iter=200, tol=1e-6, fix_nu=None, jitter=1e-6, verbose=False)
print("mu:", mu)
print("nu:", nu)
print("Sigma:\n", Sigma)
print("Precision:\n", np.linalg.pinv(Sigma))

mu: [0.0026712  0.00376902 0.00266057 0.00519398 0.00399088]
nu: 96.80952131922479
Sigma:
 [[0.98267939 0.33956943 0.36533814 0.23940811 0.30947462]
 [0.33956943 0.97688458 0.38187802 0.34141243 0.32973638]
 [0.36533814 0.38187802 0.97974535 0.29317185 0.31151825]
 [0.23940811 0.34141243 0.29317185 0.98235591 0.24193639]
 [0.30947462 0.32973638 0.31151825 0.24193639 0.97993238]]
Precision:
 [[ 1.29840624 -0.22911066 -0.29818026 -0.09493364 -0.21473029]
 [-0.22911066  1.39839976 -0.30095673 -0.28306037 -0.23263159]
 [-0.29818026 -0.30095673  1.36532152 -0.18253577 -0.19352857]
 [-0.09493364 -0.28306037 -0.18253577  1.22319206 -0.11873941]
 [-0.21473029 -0.23263159 -0.19352857 -0.11873941  1.25740894]]


### w update 

In [92]:
def w_update( w, w_lagged, x, p, nu, LnUpdate, a, u_n, z_n, Phi_n, rho, d, eta, beta):

    Lw = L_from_w(w)

    # can be replaced by an einsum

    S_sum = 0
    for t in range(0,winLen):
        S_sum += ( np.outer(x[t,:],x[t,:]) )/( x[t,:].T @ Lw @ x[t,:] + nu )
    C = (p + nu)/winLen
    S_tilde = C * S_sum


    aw = L_star(S_tilde + Phi_n + rho*(Lw - LnUpdate) + eta*np.outer(V,V.T) )
    bw = -u_n - rho*( u_n + np.multiply(a,w_lagged) ) + D_star(z_n - rho*(d - D_from_w(w) ))
    cw = (1 - rho/(rho*(4*rho -1) ) ) * w - 1/( rho*(4*rho - 1) ) * (aw + bw)
    cth = np.sqrt((2*beta)/(rho*(4*rho - 1))) * np.repeat(1,p*(p-1)/2)

    wUpdate = np.multiply((cw > cth),cw)
    return wUpdate

wUpdate = w_update(w, w_lagged, x, p, nu, LnUpdate, a, u_n, z_n, Phi_n, rho, d, eta, beta)
wUpdate

array([0.27151178, 0.39734725, 0.14655057, 0.31310886, 0.30801476,
       0.37423491, 0.26768746, 0.25462155, 0.23495535, 0.28050103])

### u update

In [93]:
def softThresh(v,thr):
    temp = abs(v) - thr
    temp = temp*(temp>0)
    return(np.sign(v)*temp)


def u_update(wUpdate,a,w_lagged,mu_vec,rho,alpha):
    # Update u
    u  = wUpdate - a*w_lagged - mu_vec/rho
    thr = alpha/(rho)
    uUpdate = softThresh(u, thr)
    return uUpdate

uUpdate = u_update(wUpdate,a,w_lagged,mu_vec,rho,alpha)
uUpdate

array([0.2624634 , 0.38829888, 0.1375022 , 0.30406049, 0.29896639,
       0.36518653, 0.25863908, 0.24557317, 0.22590698, 0.27145265])

### X update

In [55]:
pass

### a update

In [96]:

def a_update(a,wUpdate,uUpdate,mu_vec,gamma,rho,w_lagged):
    # update a
    f_temp = wUpdate - uUpdate - mu_vec/rho
    f_temp[f_temp<0] = 0
    idx = w_lagged > 0
    thr = gamma/(rho* w_lagged[idx]^2)
    a[idx] = softThresh(f_temp[idx]/w_lagged[idx], thr)
    a[~idx] = 0
    aUpdate = a.copy()
    return aUpdate

aUpdate = a_update( a, wUpdate, uUpdate, mu_vec, gamma, rho, w_lagged)

aUpdate

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### V update

In [97]:
def V_update(wUpdate):

    LwUpdate = L_from_w(wUpdate)

    # update V
    eigenVals, eigenVectors = np.linalg.eigh(LwUpdate) # Returns in Ascending Order
    vUpdate = eigVecs[:, :k]   # smallest k eigenvalues
    return vUpdate

vUpdate = V_update(wUpdate)
vUpdate

array([[0.4472136],
       [0.4472136],
       [0.4472136],
       [0.4472136],
       [0.4472136]])

### Dual Variable Update

In [98]:
def dual_update( wUpdate, LnUpdate, aUpdate, uUpdate):
    # update Phi
    R1 =  L_from_w(wUpdate) - LnUpdate
    Phi_nUpdate = Phi_n + rho * R1

    # update mu
    R0 = uUpdate - wUpdate + np.multiply(aUpdate,w_lagged) 
    mu_vecUpdate = mu_vec + rho * R0

    # update z
    R2 = D_from_w(wUpdate) - d
    z_nUpdate = z_n + rho * R2

    return Phi_nUpdate, mu_vecUpdate, z_nUpdate

Phi_nUpdate, mu_vecUpdate, z_nUpdate = dual_update(wUpdate,LnUpdate,aUpdate,uUpdate)

print(Phi_nUpdate)
print(mu_vecUpdate)
print(z_nUpdate)

[[-0.31733882  0.10772709  0.04547857  0.11350114  0.05063203]
 [ 0.10772709 -0.26700906  0.08451392  0.00149848  0.07326956]
 [ 0.04547857  0.08451392 -0.27566696  0.0596184   0.08605608]
 [ 0.11350114  0.00149848  0.0596184  -0.19922361  0.0246056 ]
 [ 0.05063203  0.07326956  0.08605608  0.0246056  -0.23456327]]
[-0.00904837 -0.00904837 -0.00904837 -0.00904837 -0.00904837 -0.00904837
 -0.00904837 -0.00904837 -0.00904837 -0.00904837]
[0.12851846 0.2214489  0.19493891 0.05590805 0.0962527 ]


In [ ]:
L_n = LnUpdate
w = wUpdate

u_n = uUpdate

# x Pass

a = aUpdate
V = vUpdate

Phu_n = Phi_nUpdate
mu_vec = mu_vecUpdate
z_n = z_nUpdate



array([0.27151178, 0.39734725, 0.14655057, 0.31310886, 0.30801476,
       0.37423491, 0.26768746, 0.25462155, 0.23495535, 0.28050103])